In [1]:
from pathlib import Path
from rich import print

### Load configuration


In [2]:
from w2t_bkin import config, ingest, sync, events, nwb, domain

In [3]:
cfg = config.load_config("tests/fixtures/configs/valid_config.toml")
print(cfg)

Config(
    project=ProjectConfig(name='w2t-bkin-pipeline'),
    paths=PathsConfig(
        raw_root='tests/fixtures/data/raw',
        intermediate_root='data/interim',
        output_root='data/processed',
        metadata_file='session.toml',
        models_root='models'
    ),
    timebase=TimebaseConfig(
        source='nominal_rate',
        mapping='nearest',
        jitter_budget_s=0.01,
        offset_s=0.0,
        ttl_id=None,
        neuropixels_stream=None
    ),
    acquisition=AcquisitionConfig(concat_strategy='ffconcat'),
    verification=VerificationConfig(mismatch_tolerance_frames=0, warn_on_mismatch=False),
    bpod=BpodConfig(parse=True),
    video=VideoConfig(transcode=TranscodeConfig(enabled=True, codec='h264', crf=20, preset='fast', keyint=15)),
    nwb=NWBConfig(
        link_external_video=True,
        lab='Lab Name',
        institution='Institution Name',
        file_name_template='{session.id}.nwb',
        session_description_template='Session {session.id} on {session.date}'
    ),
    qc=QCConfig(generate_report=True, out_template='qc/{session.id}', include_verification=True),
    logging=LoggingConfig(level='INFO', structured=False),
    labels=LabelsConfig(
        dlc=DLCConfig(run_inference=False, model='model.pb'),
        sleap=SLEAPConfig(run_inference=False, model='sleap.h5')
    ),
    facemap=FacemapConfig(run_inference=False, ROIs=['face', 'left_eye', 'right_eye'])
)

In [4]:
session = config.load_session("tests/fixtures/data/raw/Session-000001/session.toml")
print(session)

Session(
    session=SessionMetadata(
        id='Session-000001',
        subject_id='mouse_001',
        date='2025-01-01',
        experimenter='Borja Esteban',
        description='Test session',
        sex='M',
        age='P60',
        genotype='WT'
    ),
    bpod=BpodSession(
        path='Bpod/*.mat',
        order='name_asc',
        continuous_time=False,
        trial_types=[
            BpodTrialType(
                trial_type=1,
                description='Active wisker touch trials',
                sync_signal='W2T_Audio',
                sync_ttl='ttl_cue'
            ),
            BpodTrialType(
                trial_type=2,
                description='Passive wisker touch trials',
                sync_signal='A2L_Audio',
                sync_ttl='ttl_cue'
            ),
            BpodTrialType(
                trial_type=3,
                description='Microstim trials',
                sync_signal='Microstim',
                sync_ttl='ttl_cue'
            )
        ]
    ),
    TTLs=[
        TTL(id='ttl_hitmiss', description='Hit/Miss signal from behavior', paths='TTLs/*nidq.xa_1_0*.txt'),
        TTL(id='ttl_camera', description='Camera sync pulses', paths='TTLs/*nidq.xa_7_0*.txt'),
        TTL(id='ttl_cue', description='Cue signal from behavior', paths='TTLs/*nidq.xia_3_0*.txt')
    ],
    cameras=[
        Camera(
            id='cam0',
            description='Top-down camera view of mouse',
            paths='Video/top/*.avi',
            order='name_asc',
            ttl_id='ttl_camera'
        ),
        Camera(
            id='cam1',
            description='Left eye pupil camera',
            paths='Video/pupil_left/*.avi',
            order='name_asc',
            ttl_id='ttl_camera'
        )
    ],
    session_dir='/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001'
)

### Ingest and verify


In [5]:
manifest = ingest.discover_files(cfg, session)
manifest = ingest.populate_manifest_counts(manifest)
print(manifest)

Manifest(
    session_id='Session-000001',
    cameras=[
        ManifestCamera(
            camera_id='cam0',
            ttl_id='ttl_camera',
            video_files=[
                '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/Video/top/cam0_2025-01-01-00-00-00.avi
'
            ],
            frame_count=8580,
            ttl_pulse_count=8580
        ),
        ManifestCamera(
            camera_id='cam1',
            ttl_id='ttl_camera',
            video_files=[
                '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/Video/pupil_left/cam1_2025-01-01-00-00
-00.avi'
            ],
            frame_count=8580,
            ttl_pulse_count=8580
        )
    ],
    ttls=[
        ManifestTTL(
            ttl_id='ttl_hitmiss',
            files=[
                '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/TTLs/session.nidq.xa_1_0_hitmiss.txt'
            ]
        ),
        ManifestTTL(
            ttl_id='ttl_camera',
            files=[
                '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/TTLs/session.nidq.xa_7_0_frame.txt'
            ]
        ),
        ManifestTTL(
            ttl_id='ttl_cue',
            files=[
                '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/TTLs/session.nidq.xia_3_0_cue.txt'
            ]
        )
    ],
    bpod_files=[
        '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/Bpod/session_file01.mat',
        '/home/borja/w2t-bkin/tests/fixtures/data/raw/Session-000001/Bpod/session_file02.mat'
    ]
)

In [6]:
verification = ingest.verify_manifest(manifest, tolerance=5, warn_on_mismatch=True)
print(verification)

VerificationResult(
    status='pass',
    camera_results=[
        CameraVerificationResult(
            camera_id='cam0',
            ttl_id='ttl_camera',
            frame_count=8580,
            ttl_pulse_count=8580,
            mismatch=0,
            verifiable=True,
            status='pass'
        ),
        CameraVerificationResult(
            camera_id='cam1',
            ttl_id='ttl_camera',
            frame_count=8580,
            ttl_pulse_count=8580,
            mismatch=0,
            verifiable=True,
            status='pass'
        )
    ]
)

### Parse Bpod behavioral data


In [7]:
bpod_data = events.parse_bpod(
    session_dir=Path(session.session_dir),  # ..../tests/fixtures/data/raw/Session-000001
    pattern=session.bpod.path,  # 'Bpod/*.mat'
    order="name_asc",
    continuous_time=False,
)

In [8]:
trials = events.extract_trials(bpod_data)
print(trials)

[
    Trial(trial_number=1, trial_type=2, start_time=0.1438, stop_time=11.3157, outcome=<TrialOutcome.HIT: 'hit'>),
    Trial(trial_number=2, trial_type=2, start_time=11.3443, stop_time=22.0135, outcome=<TrialOutcome.HIT: 'hit'>),
    Trial(
        trial_number=3,
        trial_type=1,
        start_time=22.0374,
        stop_time=30.4916,
        outcome=<TrialOutcome.MISS: 'miss'>
    )
]

In [9]:
bevents = events.extract_behavioral_events(bpod_data)
print([x for x in bevents if x.metadata["trial_number"] == 3.0])

[
    TrialEvent(event_type='Flex1Trig2', timestamp=22.0377, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.0665, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.070400000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.218500000000002, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Flex1Trig1', timestamp=22.2347, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.2787, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.363100000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.5394, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.5789, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.6841, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.816000000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.822100000000002, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.826400000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.8504, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.858400000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=22.861900000000002, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=22.9525, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=23.073300000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=23.178700000000003, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=23.271900000000002, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=23.3988, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=23.4002, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=23.4004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=23.4264, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=28.4816, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Flex1Trig2', timestamp=28.481700000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=30.4816, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=30.491600000000002, metadata={'trial_number': 3.0})
]

In [10]:
# summary = events.create_event_summary(session, trials, bevents)
# print(summary)

### Import pose data


### Import facemap data


### Transcode videos for better storage


### Synchronize to reference timebase


In [11]:
ttl_patterns = {ttl.id: ttl.paths for ttl in session.TTLs}
ttl_pulses = sync.get_ttl_pulses(ttl_patterns, session.session_dir)
trial_offsets, warnings = sync.align_bpod_trials_to_ttl(
    trial_type_configs=session.bpod.trial_types,
    bpod_data=bpod_data,
    ttl_pulses=ttl_pulses,
)

TTL channel 'ttl_hitmiss' has 4 unused pulses
TTL channel 'ttl_camera' has 8580 unused pulses
TTL channel 'ttl_camera' has 8580 unused pulses


In [12]:
trials = events.extract_trials(bpod_data, trial_offsets)
print(trials)

[
    Trial(
        trial_number=1,
        trial_type=2,
        start_time=15.501826000000003,
        stop_time=26.673726000000002,
        outcome=<TrialOutcome.HIT: 'hit'>
    ),
    Trial(
        trial_number=2,
        trial_type=2,
        start_time=26.702474,
        stop_time=37.371674,
        outcome=<TrialOutcome.HIT: 'hit'>
    ),
    Trial(
        trial_number=3,
        trial_type=1,
        start_time=37.395712,
        stop_time=45.849911999999996,
        outcome=<TrialOutcome.MISS: 'miss'>
    )
]

In [13]:
bevents_aligned = events.extract_behavioral_events(bpod_data, trial_offsets)
print([x for x in bevents_aligned if x.metadata["trial_number"] == 3.0])

[
    TrialEvent(event_type='Flex1Trig2', timestamp=37.396012, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=37.424812, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=37.428712000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=37.576812000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Flex1Trig1', timestamp=37.593012, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=37.637012, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=37.721412, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=37.897712, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=37.937212, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.042412, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.174312, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.180412000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.184712000000005, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.208712, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.216712, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.220212000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.310812, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.431612, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.537012000000004, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.630212, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.757112, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.758511999999996, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1High', timestamp=38.758712, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='BNC1Low', timestamp=38.784712, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=43.839912, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Flex1Trig2', timestamp=43.840012, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=45.839912, metadata={'trial_number': 3.0}),
    TrialEvent(event_type='Tup', timestamp=45.849912, metadata={'trial_number': 3.0})
]

### Assemble NWB


In [14]:
output_dir = Path("data/external/Session-000001")
provenance = {
    "config_hash": "abc123",
    "session_hash": "def456",
    "software": {"name": "w2t_bkin", "version": "0.1.0"},
    "timebase": {"source": "nominal_rate"},
}
nwb_path = nwb.assemble_nwb(manifest, cfg, provenance, output_dir)

/home/borja/w2t-bkin/.venv/lib/python3.10/site-packages/pynwb/file.py:492: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


### Validate and QC (Phase 5 - not yet implemented)


In [15]:
validate.run_nwbinspector(nwb_path)
qc.render_qc_report(nwb_path, verification, ...)

NameError: name 'validate' is not defined